In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# -----------------------------
# Read bronze (choose one)
# -----------------------------
bronze_path = "../data/bronze/power_load.parquet"   # <-- update filename if needed
df = pd.read_parquet(bronze_path)

# If you're using CSV instead:
# bronze_path = "data/bronze/power_load_ts_1s.csv"
# df = pd.read_csv(bronze_path)

# -----------------------------
# Ensure types
# -----------------------------
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="raise")
df = df.sort_values("timestamp")

# -----------------------------
# Create day label + seconds-of-day
# -----------------------------
df["date"] = df["timestamp"].dt.normalize()  # midnight of that date
df["second_of_day"] = (df["timestamp"] - df["date"]).dt.total_seconds().astype(int)

# -----------------------------
# Select which days to plot (first 6 by date)
# -----------------------------
unique_dates = df["date"].drop_duplicates().sort_values()
selected_dates = unique_dates.iloc[:6]   # like MATLAB [1:6]
resolution = 1

# x-axis in hours
time_hour = np.arange(0, 24 * 3600, resolution) / 3600

fig = go.Figure()

for d in selected_dates:
    day_df = df[df["date"] == d].sort_values("second_of_day")

    # ensure we plot at the requested resolution
    day_df = day_df.iloc[::resolution]

    fig.add_trace(
        go.Scatter(
            x=day_df["second_of_day"].to_numpy() / 3600,
            y=day_df["load"].to_numpy() / 1e3,   # convert to kW like your original
            mode="lines",
            name=str(d.date()),  # legend label
        )
    )

fig.update_layout(
    title=f"Daily power/demand data over {len(selected_dates)} day(s)",
    xaxis_title=f"time [hours in {resolution} sec. resolution]",
    yaxis_title="power/demand [kW]",
    template="plotly_white",
)

fig.show()
